In [3]:
import cv2
import mediapipe as mp
import numpy as np

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# 비디오 파일 열기
cap = cv2.VideoCapture(r'G:\내 드라이브\Sign_Remaster\Sign_Language_Remaster\test_hands.mp4')  # 여기서 'video.mp4'는 사용할 영상 파일명입니다.
data = []

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break

    # 영상을 RGB로 변환하여 Mediapipe에 전달
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)
    

    if results.multi_hand_landmarks:
        print(f"Number of hands detected: {len(results.multi_hand_landmarks)}")
        all_d = []
        for res in results.multi_hand_landmarks:
            joint = np.zeros((21, 4))  # 21개의 관절에 대한 정보 저장 (x, y, z, visibility)

            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

            # Compute angles between joints
            v1 = joint[[0, 1, 2, 3, 0, 5, 6, 7, 0, 9, 10, 11, 0, 13, 14, 15, 0, 17, 18, 19], :3]
            v2 = joint[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], :3]
            v = v2 - v1
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            angle = np.arccos(np.einsum('nt,nt->n',
                                        v[[0, 1, 2, 4, 5, 6, 8, 9, 10, 12, 13, 14, 16, 17, 18], :],
                                        v[[1, 2, 3, 5, 6, 7, 9, 10, 11, 13, 14, 15, 17, 18, 19], :]))

            angle = np.degrees(angle)
            angle_label = np.array([angle], dtype=np.float32)
            angle_label = np.append(angle_label, 1)  # Assuming idx is 1, change it accordingly

            d = np.concatenate([joint.flatten(), angle_label])
            all_d.append(d)
        data.append(np.array(all_d))

            # mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
    else:
        print("No hands detected in this frame")
    cv2.imshow('Hand Vector Data', img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 저장된 데이터 확인
data = np.array(data)
print("Shape of data:", data.shape)

# 데이터를 파일로 저장 (예: CSV 형태로)
np.savetxt('hand_vector_data.csv', data, delimiter=',')  # 변경 가능한 파일명 및 형식입니다.

cap.release()
cv2.destroyAllWindows()


No hands detected in this frame
No hands detected in this frame
No hands detected in this frame
No hands detected in this frame
No hands detected in this frame
No hands detected in this frame
No hands detected in this frame
No hands detected in this frame
No hands detected in this frame
No hands detected in this frame
No hands detected in this frame
No hands detected in this frame
No hands detected in this frame
No hands detected in this frame
No hands detected in this frame
No hands detected in this frame
No hands detected in this frame
No hands detected in this frame
No hands detected in this frame
No hands detected in this frame
No hands detected in this frame
No hands detected in this frame
No hands detected in this frame
No hands detected in this frame
Number of hands detected: 1
Number of hands detected: 1
Number of hands detected: 1
Number of hands detected: 1
Number of hands detected: 1
Number of hands detected: 2
Number of hands detected: 1
Number of hands detected: 2
Number o

In [3]:
### 동영상으로 벡터 추출
import cv2
import mediapipe as mp
import numpy as np
import time, os

seq_length = 30
secs_for_action = 30

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

video_path = 'test_hands.mp4'  # 동영상 파일 경로

action = input("녹화할 동작을 입력하세요: ")  # 사용자 입력으로 동작 받기

cap = cv2.VideoCapture(video_path)  # 동영상 파일을 VideoCapture 객체로 열기

created_time = int(time.time())
os.makedirs('dataset', exist_ok=True)

while cap.isOpened():
    data = []

    start_time = time.time()

    while time.time() - start_time < secs_for_action:
        ret, img = cap.read()

        if not ret:  # 동영상이 끝나면 종료
            break

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = hands.process(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        if result.multi_hand_landmarks is not None:
            for res in result.multi_hand_landmarks:
                joint = np.zeros((21, 4))
                for j, lm in enumerate(res.landmark):
                    joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

                v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3]
                v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3]
                v = v2 - v1
                v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                angle = np.arccos(np.einsum('nt,nt->n',
                    v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                    v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))
                angle = np.degrees(angle)

                angle_label = np.array([angle], dtype=np.float32)
                angle_label = np.append(angle_label, action)  # 사용자가 입력한 동작 사용

                d = np.concatenate([joint.flatten(), angle_label])
                data.append(d)

        if not ret:  # 동영상이 끝나면 종료
            break

    if len(data) == 0:  # 데이터가 없으면 종료
        break

    data = np.array(data)
    np.save(os.path.join('dataset', f'raw_{action}_{created_time}'), data)

    full_seq_data = []
    for seq in range(len(data) - seq_length):
        full_seq_data.append(data[seq:seq + seq_length])

    full_seq_data = np.array(full_seq_data)
    np.save(os.path.join('dataset', f'seq_{action}_{created_time}'), full_seq_data)

    break  # 여기서 영상 한 번만 실행하도록 설정

cap.release()  # 비디오 캡처 객체 해제
cv2.destroyAllWindows()  # 모든 창 닫기


In [1]:
# 한손학습
import cv2
import mediapipe as mp
import numpy as np
import time, os

actions = ['NOISE']
seq_length = 30
secs_for_action = 30

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

created_time = int(time.time())
os.makedirs('dataset', exist_ok=True)

while cap.isOpened():
    for idx, action in enumerate(actions):
        data = []
        ret, img = cap.read()
        img = cv2.flip(img, 1)

        cv2.putText(img, f'Waiting for collecting {action.upper()} action...', org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
        cv2.imshow('img', img)
        cv2.waitKey(3000)

        start_time = time.time()

        while time.time() - start_time < secs_for_action:
            ret, img = cap.read()

            img = cv2.flip(img, 1)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            result = hands.process(img)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            if result.multi_hand_landmarks is not None:
                for res in result.multi_hand_landmarks:
                    joint = np.zeros((21, 4))
                    for j, lm in enumerate(res.landmark):
                        joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

                    # Compute angles between joints
                    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                    v = v2 - v1 # [20, 3]
                    # Normalize v
                    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                    # Get angle using arcos of dot product
                    angle = np.arccos(np.einsum('nt,nt->n',
                        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

                    angle = np.degrees(angle) # Convert radian to degree

                    angle_label = np.array([angle], dtype=np.float32)
                    angle_label = np.append(angle_label, idx)

                    d = np.concatenate([joint.flatten(), angle_label])
                    print(d.shape,'d')
                    data.append(d)

                    mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

            cv2.imshow('img', img)
            if cv2.waitKey(1) == ord('q'):
                break

        data = np.array(data)
        print(action, data.shape)
        np.save(os.path.join('dataset', f'raw_{action}_{created_time}'), data)

        # Create sequence data
        full_seq_data = []
        for seq in range(len(data) - seq_length):
            full_seq_data.append(data[seq:seq + seq_length])

        full_seq_data = np.array(full_seq_data)
        print(action, full_seq_data.shape)
        np.save(os.path.join('dataset', f'seq_{action}_{created_time}'), full_seq_data)
    break

NOISE (437, 100)
NOISE (407, 30, 100)


: 

In [32]:
# 양손학습
import cv2
import mediapipe as mp
import numpy as np
import time, os

   
actions = ['test']
seq_length = 30
secs_for_action = 30
target_frame = 450
frame_count = 0

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

created_time = int(time.time())
os.makedirs('dataset', exist_ok=True)

while cap.isOpened():
    for idx, action in enumerate(actions):
        data = []
        ret, img = cap.read()
        img = cv2.flip(img, 1)

        cv2.putText(img, f'Waiting for collecting {action.upper()} action...', org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
        cv2.imshow('img', img)
        cv2.waitKey(3000)
        angles = []
        while frame_count < target_frame:
            frame_count+=1
            ret, img = cap.read()
            img = cv2.flip(img, 1)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            result = hands.process(img)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            
            if result.multi_hand_landmarks is not None:
                two_hand=[]
                for res in result.multi_hand_landmarks:
                    joint = np.zeros((21, 4))
                    for j, lm in enumerate(res.landmark):
                        joint[j] = [lm.x, lm.y, lm.z, lm.visibility]
                        print([lm.x, lm.y, lm.z, lm.visibility])
                    # Compute angles between joints
                    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                    v = v2 - v1 # [20, 3]
                    # Normalize v
                    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
                    
                    # Get angle using arcos of dot product
                    angle = np.arccos(np.einsum('nt,nt->n',
                        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

                    angle = np.degrees(angle) # Convert radian to degree
                    two_hand.extend(angle)
                    mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
                angles.append(two_hand)
            cv2.imshow('img', img)
            if cv2.waitKey(1) == ord('q'):
                break

        data = np.array(angles)
        # Create sequence data
        full_seq_data = []
        for seq in range(len(data) - seq_length):
            full_seq_data.append(data[seq:seq + seq_length])

        full_seq_data = np.array(full_seq_data)
        print(action, full_seq_data.shape)
        np.save(os.path.join('dataset', f'seq_{action}_{created_time}'), full_seq_data)
    cv2.destroyAllWindows()
    break

[0.4832397699356079, 0.8531175851821899, 1.967736409369536e-07, 0.0]
[0.4709940552711487, 0.7323461174964905, -0.005005279555916786, 0.0]
[0.495810329914093, 0.6426513195037842, -0.0033056875690817833, 0.0]
[0.5361775755882263, 0.587568998336792, 0.0004141275130677968, 0.0]
[0.5635665059089661, 0.544547975063324, 0.005349738523364067, 0.0]
[0.5691831707954407, 0.6583961844444275, 0.004270539619028568, 0.0]
[0.6230053305625916, 0.6147384643554688, 0.007985229603946209, 0.0]
[0.6516451835632324, 0.5940359234809875, 0.00825849175453186, 0.0]
[0.6736471652984619, 0.5761058330535889, 0.007969670929014683, 0.0]
[0.5915078520774841, 0.716525137424469, 0.005968508310616016, 0.0]
[0.6457408666610718, 0.6694536209106445, 0.015093500725924969, 0.0]
[0.6753066778182983, 0.645378828048706, 0.016495749354362488, 0.0]
[0.6960127353668213, 0.628986120223999, 0.015557835809886456, 0.0]
[0.600213348865509, 0.7737850546836853, 0.006642505992203951, 0.0]
[0.6511632204055786, 0.7335428595542908, 0.01172819

C:\Users\2580j\AppData\Local\Temp\ipykernel_21428\3454852833.py:72: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.array(angles)


In [2]:
import numpy as np

zeros_array = np.zeros(100)
print(zeros_array.shape)  # 출력: (100,)


(100,)


In [9]:
np.concatenate([[1,2],[3,4]])

array([1, 2, 3, 4])

In [1]:
# 한손학습
import cv2
import mediapipe as mp
import numpy as np
import time, os

actions = [
    'hello',
    'bread_house',
    'lunch',
    'NOISE'
]
seq_length = 30
secs_for_action = 30

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

created_time = int(time.time())
print(created_time)
os.makedirs(f'dataset/{created_time}', exist_ok=True)

while cap.isOpened():
    for idx, action in enumerate(actions):
        print(action)
        data = []
        ret, img = cap.read()
        img = cv2.flip(img, 1)

        cv2.putText(img, f'Waiting for collecting {action.upper()} action...', org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
        cv2.imshow('img', img)
        cv2.waitKey(3000)

        start_time = time.time()

        while time.time() - start_time < secs_for_action:
            ret, img = cap.read()

            img = cv2.flip(img, 1)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            result = hands.process(img)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            if result.multi_hand_landmarks is not None:
                da = [] 
                if len(result.multi_hand_landmarks) == 2:
                    d = []
                    for res in result.multi_hand_landmarks:  # res 잡힌 만큼 (max 손 개수 이하)
                        mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
                        joint = np.zeros((21, 3))
                        for j, lm in enumerate(res.landmark):
                            joint[j] = [lm.x, lm.y, lm.z]

                        # Compute angles between joints
                        v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                        v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                        v = v2 - v1 # [20, 3]
                        # Normalize v
                        v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                        # Get angle using arcos of dot product
                        angle = np.arccos(np.einsum('nt,nt->n',
                            v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                            v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]\

                        angle = np.degrees(angle) # Convert radian to degree
                        ############################################
                        angle = np.array([angle], dtype=np.float32)
                        d.append(np.concatenate([joint.flatten(),angle.flatten()]))
                    da.append([np.concatenate(d)])
                    data.append(np.concatenate(da))        
                    
                    # da.append([np.concatenate(d),idx])
                elif len(result.multi_hand_landmarks)==1:
                    d= []
                    for res in result.multi_hand_landmarks:  # res 잡힌 만큼 (max 손 개수 이하)
                        mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
                        joint = np.zeros((21, 3))
                        for j, lm in enumerate(res.landmark):
                            joint[j] = [lm.x, lm.y, lm.z]

                        # Compute angles between joints
                        v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                        v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                        v = v2 - v1 # [20, 3]
                        # Normalize v
                        v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                        # Get angle using arcos of dot product
                        angle = np.arccos(np.einsum('nt,nt->n',
                            v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                            v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]\

                        angle = np.degrees(angle) # Convert radian to degree

                        angle = np.array([angle], dtype=np.float32)
                        d.append(np.concatenate([joint.flatten(),angle.flatten()]))
                        
                    d.append(np.zeros_like(d[0]))
                    da.append([np.concatenate(d)])  
                    data.append(np.concatenate(da))        

                
                # mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

            cv2.imshow('img', img)
            if cv2.waitKey(1) == ord('q'):
                break

        data = np.array(data)
        data = data.reshape(data.shape[0],data.shape[-1])
        full_seq_data = []
        # print(data.shape)
        for seq in range(len(data) - seq_length):
            full_seq_data.append(data[seq:seq + seq_length])

        full_seq_data = np.array(full_seq_data)
        print(action, full_seq_data.shape, data.shape)
        # if fs == 0:
        #     fs = full_seq_data
        #     da = data
        if len(full_seq_data.shape) ==3 :
            np.save(os.path.join(f'dataset/{created_time}', f'raw_{action}'), data)
            np.save(os.path.join(f'dataset/{created_time}', f'seq_{action}_{full_seq_data.shape}'), full_seq_data)
            print(action,'saved')
    cv2.destroyAllWindows()
    break
if os.path.exists(f'dataset/{created_time}') and not os.listdir(f'dataset/{created_time}'):
    os.rmdir(f'dataset/{created_time}')  
    print(f"{f'dataset/{created_time}'} 빈폴더 삭제")

1704553953
hello
hello (654, 30, 156) (684, 156)
hello saved
bread_house
bread_house (504, 30, 156) (534, 156)
bread_house saved
lunch
lunch (592, 30, 156) (622, 156)
lunch saved
NOISE
NOISE (636, 30, 156) (666, 156)
NOISE saved


In [15]:
f'({full_seq_data.shape})'

'((1053, 30))'

In [5]:
a = [1,2,3,4]
b = [0,0,0,0]
c = [np.array(a),np.array(b)]
d= np.array(np.concatenate(c))
d[1:3]

array([2, 3])